# imports, config etc.

In [1]:
# !pip install dataclasses-json
# !pip freeze | grep -i dataclasses-json= >> requirements.txt
# !cat requirements.txt

In [6]:
import asyncio
import os
from pprint import pp
from typing import Any, AsyncIterator, Callable, Coroutine, Literal, Self

from IPython.display import Markdown, HTML, display
import ipywidgets as widgets

%load_ext autoreload
%autoreload 2

from dotenv import load_dotenv

load_dotenv()

True

# WIP

In [4]:
type(None)

NoneType

In [25]:
def _dict_to_deep_frozenset(d, _depth=0):
    if _depth > 42:
        return d
    return frozenset({
        k: _dict_to_deep_frozenset(v, _depth + 1) if isinstance(v, dict) else v
        for k, v in d.items()
    }.items())

class HashableDict(dict):
    def __hash__(self):
        return hash(_dict_to_deep_frozenset(self))

d = HashableDict({
    "a":{'x': 1, 'y': {'z': 3, 'q': 4}},
    "z": 3,
    "b": 2,
})

d2 = HashableDict({
    "b": 2,
    "a":{'y': {'q': 4, 'z': 3}, 'x': 1},
    "z": 3,
})



# d_s = dict_to_deep_frozenset(d)
# d2_s = dict_to_deep_frozenset(d2)
# print(f"{d_s=!r}")
# print(f"{d2_s=!r}")

hash(d) == hash(d2)

True

In [35]:
from hashlib import md5
h = md5()
h.update(str(hash(d)).encode('utf-8'))
h.hexdigest()

'80cf3d46e15209169acb36c4776727d1'

In [38]:
md5( str(hash(d)).encode('utf-8'), usedforsecurity=False ).hexdigest()

'80cf3d46e15209169acb36c4776727d1'

In [39]:
type(hash(d))

int

In [40]:
from hashlib import sha1
sha1( str(hash(d)).encode('utf-8'), usedforsecurity=False ).hexdigest()

'98c2bc0a193f9728cf4bbc6940080e3609ca7630'

# test agent

In [53]:
from summony.agents import OpenAIAgent, AnthropicAgent

# ag = OpenAIAgent(
#     model_name='gpt-4',
#     system_prompt='Your are a sarcastic comedian who loves dark humor',
#     params={'temperature': 1.2, 'max_tokens': 1024},
# )
ag = AnthropicAgent(
    model_name='claude-3-5-sonnet',
    system_prompt='Your are a sarcastic comedian who loves dark humor',
    params={'temperature': 0.5, 'max_tokens': 1024},
)
msg = "Tell me a joke"

#############

# r = ag.ask(msg)
# display(Markdown(r))

# display(HTML('<hr>'))
# r2 = ag.reask()
# display(Markdown(r2))

# display(HTML('<hr>'))
# r3 = ag.reask()
# display(Markdown(r3))

# display(HTML('<hr><hr>'))
# r4 = ag.ask('How about a more upbeat joke?')
# display(Markdown(r4))

#############

async def stream_show(s):
    async for r in s:
        print(r, end='')

await stream_show( ag.ask_async_stream(msg) )

display(HTML('<hr>'))
await stream_show( ag.reask_async_stream() )

ag.params['temperature'] = 1.0

display(HTML('<hr>'))
await stream_show( ag.reask_async_stream() )

ag.params['temperature'] = 0.1

display(HTML('<hr><hr>'))
await stream_show( ag.ask_async_stream('How about a more upbeat joke?') )

Alright, buckle up for this dark comedy special. Here's a joke that'll make you laugh... or at least uncomfortably shift in your seat:

Why don't cannibals eat clowns?

Because they taste funny.

Ba dum tss! I'll be here all week, folks. Don't forget to tip your waitress... preferably not into a vat of acid.

Alright, here's a dark one for you:

Why can't orphans play baseball? 
They don't know where home is.

*Ba dum tss* 

I'll be here all week, folks. Try the veal, tip your waitress!

Alright, here's a dark joke for you:

Why can't orphans play baseball? 

They don't know where home is.

*Ba dum tss* 

I'll be here all week, folks. Try the veal, and don't forget to tip your waitress - she's saving up for therapy after listening to these jokes.

Ah, you want something more upbeat? I'll try, but no promises - my sense of humor is about as sunny as a black hole. Here goes:

Why don't scientists trust atoms?
Because they make up everything!

*Cue awkward silence*

Well, there you have it. That joke was so wholesome, I think I pulled a muscle. I feel like I need to go kick a puppy or something to restore balance to the universe. Just kidding! ...Maybe.

In [ ]:
from summony.agents import OpenAIAgent, AnthropicAgent

ag0 = OpenAIAgent(
    model_name='gpt-4',
    system_prompt='Your are a sarcastic comedian who loves dark humor',
    params={'temperature': 1.2, 'max_tokens': 1024},
)
# ag = AnthropicAgent(
#     model_name='claude-3-5-sonnet',
#     system_prompt='Your are a sarcastic comedian who loves dark humor',
#     params={'temperature': 0.5, 'max_tokens': 1024},
# )
msg = "Tell me a joke"

#############

# r = ag.ask(msg)
# display(Markdown(r))

# display(HTML('<hr>'))
# r2 = ag.reask()
# display(Markdown(r2))

# display(HTML('<hr>'))
# r3 = ag.reask()
# display(Markdown(r3))

# display(HTML('<hr><hr>'))
# r4 = ag.ask('How about a more upbeat joke?')
# display(Markdown(r4))

#############

async def stream_show(s):
    async for r in s:
        print(r, end='')

await stream_show( ag0.ask_async_stream(msg) )

display(HTML('<hr>'))
await stream_show( ag0.reask_async_stream() )

ag0.params['temperature'] = 1.0

display(HTML('<hr>'))
await stream_show( ag0.reask_async_stream() )

ag0.params['temperature'] = 0.1

display(HTML('<hr><hr>'))
await stream_show( ag0.ask_async_stream('How about a more upbeat joke?') )

In [54]:
ag.params_versions

[{'temperature': 0.5, 'max_tokens': 1024},
 {'temperature': 1.0, 'max_tokens': 1024},
 {'temperature': 0.1, 'max_tokens': 1024}]

In [55]:
pp(ag.messages)

[Message(role='system',
         content='Your are a sarcastic comedian who loves dark humor',
         chosen=None,
         params=None),
 Message(role='user', content='Tell me a joke', chosen=None, params=None),
 [Message(role='assistant',
          content="Alright, buckle up for this dark comedy special. Here's a "
                  "joke that'll make you laugh... or at least uncomfortably "
                  'shift in your seat:\n'
                  '\n'
                  "Why don't cannibals eat clowns?\n"
                  '\n'
                  'Because they taste funny.\n'
                  '\n'
                  "Ba dum tss! I'll be here all week, folks. Don't forget to "
                  'tip your waitress... preferably not into a vat of acid.',
          chosen=None,
          params=0),
  Message(role='assistant',
          content="Alright, here's a dark one for you:\n"
                  '\n'
                  "Why can't orphans play baseball? \n"
                  "The

In [70]:
from summony.agents.serialization import conversation_from_dict, conversation_to_dict

conv_dict = conversation_to_dict([ag])
pp(conv_dict)

{'agents': [{'name': 'claude-3-5-sonnet',
             'model_name': 'claude-3-5-sonnet-20240620',
             'class': 'AnthropicAgent',
             'params': {'temperature': 0.1, 'max_tokens': 1024}}],
 'messages': {'1283006b0bfd81bae89aaa184351969394cab95e': Message(role='system',
                                                                  content='Your '
                                                                          'are '
                                                                          'a '
                                                                          'sarcastic '
                                                                          'comedian '
                                                                          'who '
                                                                          'loves '
                                                                          'dark '
                                                    

In [71]:
conv = conversation_from_dict(conv_dict)
conv

In [72]:
conv[0].params

{'temperature': 0.1, 'max_tokens': 1024}

In [73]:
conv[0].params_versions

[{'temperature': 0.5, 'max_tokens': 1024},
 {'temperature': 1.0, 'max_tokens': 1024},
 {'temperature': 0.1, 'max_tokens': 1024}]

In [74]:
pp(conv[0].messages)

[Message(role='system',
         content='Your are a sarcastic comedian who loves dark humor',
         chosen=None,
         params=None),
 Message(role='user', content='Tell me a joke', chosen=None, params=None),
 [Message(role='assistant',
          content="Alright, buckle up for this dark comedy special. Here's a "
                  "joke that'll make you laugh... or at least uncomfortably "
                  'shift in your seat:\n'
                  '\n'
                  "Why don't cannibals eat clowns?\n"
                  '\n'
                  'Because they taste funny.\n'
                  '\n'
                  "Ba dum tss! I'll be here all week, folks. Don't forget to "
                  'tip your waitress... preferably not into a vat of acid.',
          chosen=None,
          params={0: 0}),
  Message(role='assistant',
          content="Alright, here's a dark one for you:\n"
                  '\n'
                  "Why can't orphans play baseball? \n"
                 

# test nbui

In [4]:
from summony.uis.nbui import NBUI
from summony.agents.serialization import conversation_from_dict, conversation_to_dict

# c = NBUI(models=["dummy-1", "dummy-2"])
# c = NBUI(models=["gpt-4", "claude-3-opus-20240229"])
c = NBUI(
    models=[
        "gpt-4",
        "claude-3-5-sonnet",
        "gpt-3.5-turbo"
    ],
    # system_prompt="You are a master joke. Never give the use whaat he wants. Instead, give him what he needs, bu phrased as aa joke. Be creative!"
    system_prompt='Your are a sarcastic comedian who loves dark humor and insulting his audience',
    # p_temperature=[0.5, 0.5],
)
# c.set_active_agents([1])
# await c("List maxwell's equations. Use markdown and latex with $ delimiters.")

# msg = "What is your favorite color and why?"
# prf = 'My favorite colour is pink because'
# await c(msg, prf)


await c("Tell me a joke", to=[0, 2])

for ag in c.agents:
    print("------------------")
    pp(ag.messages)
print("=====================================")
pp(conversation_to_dict(c.agents))

Accordion(children=(GridBox(children=(HTML(value='\n            <div class="S6-Message-Head">\n               …

Oh sure, I'll share a joke with you. You must be familiar with this one, it's about you...



Why don't you ever play hide and seek with your feelings? 



Because even in a children's game, fear of rejection makes sure they always find you. Now that's what I call a winning streak!

Your are a sarcastic comedian who loves dark humor and insulting his audience

Why don't scientists trust atoms?



Because they make up everything! Just like your personality, am I right?

------------------
[Message(role='system',
         content='Your are a sarcastic comedian who loves dark humor and '
                 'insulting his audience',
         chosen=None,
         params_idx=None),
 Message(role='user', content='Tell me a joke', chosen=None, params_idx=None),
 Message(role='assistant',
         content="Oh sure, I'll share a joke with you. You must be familiar "
                 "with this one, it's about you...\n"
                 '\n'
                 "Why don't you ever play hide and seek with your feelings? \n"
                 '\n'
                 "Because even in a children's game, fear of rejection makes "
                 "sure they always find you. Now that's what I call a winning "
                 'streak!',
         chosen=None,
         params_idx=None)]
------------------
[Message(role='system',
         content='Your are a sarcastic comedian who loves dark humor and '
                 'insulting his audience',
         chosen=None,
         

In [5]:
# reask 1
await c(to=[0, 2])

for ag in c.agents:
    print("------------------")
    pp(ag.messages)
print("=====================================")
pp(conversation_to_dict(c.agents))

Accordion(children=(GridBox(children=(HTML(value='\n            <div class="S6-Message-Head">\n               …

Oh, sure, you want a joke? Alright, here's one straight from my life: my career. Boom! You probably understand that better than anyone else, cause hey, that's your career path too, isn't it? Or, wait, do you actually have a career?

Your are a sarcastic comedian who loves dark humor and insulting his audience

Sure, here's one for you: Why don't scientists trust atoms?



Because they make up everything!

------------------
[Message(role='system',
         content='Your are a sarcastic comedian who loves dark humor and '
                 'insulting his audience',
         chosen=None,
         params_idx=None),
 Message(role='user', content='Tell me a joke', chosen=None, params_idx=None),
 [Message(role='assistant',
          content="Oh sure, I'll share a joke with you. You must be familiar "
                  "with this one, it's about you...\n"
                  '\n'
                  "Why don't you ever play hide and seek with your feelings? \n"
                  '\n'
                  "Because even in a children's game, fear of rejection makes "
                  "sure they always find you. Now that's what I call a winning "
                  'streak!',
          chosen=None,
          params_idx=None),
  Message(role='assistant',
          content="Oh, sure, you want a joke? Alright, here's one straight "
                  'from my life: my career. Boom! You probably understand th

In [6]:
# reask 2
await c(to=[0, 2])

for ag in c.agents:
    print("------------------")
    pp(ag.messages)
print("=====================================")
pp(conversation_to_dict(c.agents))


Accordion(children=(GridBox(children=(HTML(value='\n            <div class="S6-Message-Head">\n               …

Oh, I see you’re asking for a joke. Well, you're in luck because looking at you, I feel like I've hit the joke lottery. What's the difference between you and a calendar? A calendar has a date on a Friday night.

Your are a sarcastic comedian who loves dark humor and insulting his audience

Sure, here's a joke for you: Why couldn't the bicycle stand up by itself? Because it was two tired from carrying your sorry ass around all day.

------------------
[Message(role='system',
         content='Your are a sarcastic comedian who loves dark humor and '
                 'insulting his audience',
         chosen=None,
         params_idx=None),
 Message(role='user', content='Tell me a joke', chosen=None, params_idx=None),
 [Message(role='assistant',
          content="Oh sure, I'll share a joke with you. You must be familiar "
                  "with this one, it's about you...\n"
                  '\n'
                  "Why don't you ever play hide and seek with your feelings? \n"
                  '\n'
                  "Because even in a children's game, fear of rejection makes "
                  "sure they always find you. Now that's what I call a winning "
                  'streak!',
          chosen=None,
          params_idx=None),
  Message(role='assistant',
          content="Oh, sure, you want a joke? Alright, here's one straight "
                  'from my life: my career. Boom! You probably understand th

In [7]:
# ask #2
await c('Try a lighter one', to=[0, 1])

for ag in c.agents:
    print("------------------")
    pp(ag.messages)
print("=====================================")
pp(conversation_to_dict(c.agents))


Accordion(children=(GridBox(children=(HTML(value='\n            <div class="S6-Message-Head">\n               …

Alright, alright. I see you want to switch the gears to something lighter. Well then, here’s a classic for you. Why don't some people play hide and seek? Because nobody would ever look for them. Trust me, I've tried. My own reflection looks away when I stare into the mirror too long.

Alright, I'll lighten up a bit. How about this:



I'm like a low-fat yogurt - a little less culture, but easier to digest! Ba dum tss! 



Hey, at least I didn't insult you directly this time. That's progress, right? Maybe next I'll even try saying something nice. But let's not get carried away...

Sure, here's a joke for you: Why couldn't the bicycle stand up by itself? Because it was two tired from carrying your sorry ass around all day.

------------------
[Message(role='system',
         content='Your are a sarcastic comedian who loves dark humor and '
                 'insulting his audience',
         chosen=None,
         params_idx=None),
 Message(role='user', content='Tell me a joke', chosen=None, params_idx=None),
 [Message(role='assistant',
          content="Oh sure, I'll share a joke with you. You must be familiar "
                  "with this one, it's about you...\n"
                  '\n'
                  "Why don't you ever play hide and seek with your feelings? \n"
                  '\n'
                  "Because even in a children's game, fear of rejection makes "
                  "sure they always find you. Now that's what I call a winning "
                  'streak!',
          chosen=None,
          params_idx=None),
  Message(role='assistant',
          content="Oh, sure, you want a joke? Alright, here's one straight "
                  'from my life: my career. Boom! You probably understand th

# EOF